In [1]:
PROJECT_ID      = ""
PROJECT_REGION  = ""

GCS_BUCKET_NAME = ""

VERTEX_DATASET_NAME    = ""
VERTEX_MODEL_NAME      = ""
VERTEX_PREDICTION_NAME = ""

BQ_DATASET_NAME  = ""
BQ_TRAIN_TABLE   = ""
BQ_PREDICT_TABLE = ""

In [60]:
PROJECT_ID = "wb-ai-acltr-tbs-3-pr-a62583"
GCS_BUCKET_NAME = "bkt_b2b_wf_prediction"
PROJECT_REGION = "northamerica-northeast1"

VERTEX_DATASET_NAME = "b2b_wf_prediction_panorama"
VERTEX_MODEL_NAME = "b2b_wf_prediction_panorama"
VERTEX_PREDICTION_NAME = "b2b_wf_prediction_batch"

BQ_DATASET_NAME = "b2b_wf_prediction"
BQ_TRAIN_TABLE = "vw_wf_experiment_historical"
BQ_PREDICT_TABLE = "bq_wf_temp_predictions"
BQ_FORECAST_TABLE= "bq_wf_forecast"

TRAIN_TEST_DATA_SPLIT = "DATE('2024-07-01')"

In [3]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
import datetime

TRAINING_DATASET_BQ_PATH   = f"bq://{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_TRAIN_TABLE}"
PREDICTION_DATASET_BQ_PATH = f"bq://{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_PREDICT_TABLE}"
PREDICTION_OUTPUT_PREFIX   = f"bq://{PROJECT_ID}.{BQ_DATASET_NAME}"
BUCKET_URI = f"gs://{PROJECT_ID}_{GCS_BUCKET_NAME}"

In [4]:
aiplatform.init(
    project=PROJECT_ID, 
    staging_bucket=BUCKET_URI,
    location=PROJECT_REGION
)

In [ ]:
client = bigquery.Client(
    project=PROJECT_ID, 
    location=PROJECT_REGION
)

In [29]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Experiment:
    name: str
    model: str
    experiment_columns: list[str]
    objective: str
    forecast_horizon: int
    context_window: int
    data_granularity_unit: str
    holiday_regions: list[str]
    

In [30]:
daily_forecast_experiment = Experiment(
    name="panorama_daily_forecast",
    model="AutoML",
    experiment_columns=[
        "District",
        "Region_Type",
        "Product",
        "Product_Grp",
        "Technology",
        "Work_Order_Action",
        "Work_Order_Action_Grp",
        "Work_Force"],
    objective="minimize-rmse",
    forecast_horizon=184,
    context_window=368,
    data_granularity_unit='day',
    holiday_regions=["CA"]
)

## Select experiment

In [31]:
running_experiment = daily_forecast_experiment

## Create train data view

In [9]:
def create_series_identifier(columns):
    coalesce_parts = [f"COALESCE({column}, 'None')" for column in columns]
    separator = "' '"
    return f"CONCAT({f', {separator}, '.join(coalesce_parts)}) as Series_Identifier"

In [ ]:
time_column                   = "Appointment_Day"
time_series_identifier_column = "Series_Identifier"
target_column                 = "SWT"

FORECAST_TIMESTAMP = datetime.datetime.now()
ATTRIBUTE_COLUMNS = running_experiment.experiment_columns
ATTRIBUTE_STRING = ','.join(ATTRIBUTE_COLUMNS)

COLUMN_SPECS = {
    time_column:             "timestamp",
    target_column:           "numeric"
}

for category in ATTRIBUTE_COLUMNS:
    COLUMN_SPECS[category] = "categorical"

In [11]:
experiment_data_cte = f"""
WITH historical_table AS (
  SELECT 
    {time_column},
    {ATTRIBUTE_STRING},
    SUM({target_column}) AS {target_column}
  FROM `{BQ_DATASET_NAME}.vw_wf_historical`
  WHERE Appointment_Day < {TRAIN_TEST_DATA_SPLIT}
  GROUP BY {time_column},{ATTRIBUTE_STRING}
)"""


experiment_train_data_query = f"""
CREATE OR REPLACE VIEW `{BQ_DATASET_NAME}.{BQ_TRAIN_TABLE}` AS 
{experiment_data_cte}
SELECT 
  {create_series_identifier(ATTRIBUTE_COLUMNS)},
  {time_column},
  {ATTRIBUTE_STRING},
  {target_column}
FROM historical_table
"""

VERTEX_DATASET_NAME += f"_{running_experiment.name}"

In [ ]:
client.query_and_wait(experiment_train_data_query)

In [ ]:
dataset_list = aiplatform.TimeSeriesDataset.list(
    filter=f"display_name={VERTEX_DATASET_NAME}"
)

if len(dataset_list) == 0:
    print("... creating new dataset ... ")
    dataset = aiplatform.TimeSeriesDataset.create(
        display_name=VERTEX_DATASET_NAME,
        bq_source=[TRAINING_DATASET_BQ_PATH],
    )
else:
    print("... using existent dataset ... ")
    dataset = dataset_list[0]

In [ ]:
model_list = aiplatform.Model.list(
    filter=f"display_name={VERTEX_MODEL_NAME}"
)

if len(model_list) == 0:
    print("... training a new model ... ")
    parent_model = None
else:
    print("... using existent model ... ")
    model = model_list[0]
    print(model)
    parent_model = model.resource_name

In [15]:
training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=VERTEX_MODEL_NAME,
    optimization_objective=running_experiment.objective,
    column_specs=COLUMN_SPECS,
)

In [ ]:
model = training_job.run(
    dataset=dataset,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=ATTRIBUTE_COLUMNS,
    forecast_horizon=running_experiment.forecast_horizon,
    context_window=running_experiment.context_window,
    data_granularity_unit=running_experiment.data_granularity_unit,
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    parent_model = parent_model,
    model_display_name=VERTEX_MODEL_NAME,
    is_default_version = True,
    model_version_description = f"{running_experiment.name} model generated on {datetime.date.today().isoformat()}",
    predefined_split_column_name=None,
    holiday_regions=running_experiment.holiday_regions
)

In [17]:
date_range = f"""SELECT
    (
      DATE(DATE_TRUNC({TRAIN_TEST_DATA_SPLIT}, DAY)) + INTERVAL i DAY
    ) AS {time_column}
  FROM
    UNNEST (GENERATE_ARRAY(0, {running_experiment.forecast_horizon-1})) AS i"""

columns_dim = f"""SELECT DISTINCT
    Series_Identifier,
    {ATTRIBUTE_STRING}
  FROM `{BQ_DATASET_NAME}.{BQ_TRAIN_TABLE}`
  WHERE
      {" IS NOT NULL AND ".join(ATTRIBUTE_COLUMNS)} IS NOT NULL
"""
future_values = f"""SELECT
    h.Series_Identifier,
    CAST(d.{time_column} AS DATE) AS {time_column},
    {','.join(map(lambda x : f'h.{x}', ATTRIBUTE_COLUMNS))},
    NULL AS {target_column},
    'predicted' AS {target_column}_Type
  FROM columns_dim h,
    date_range d
  """

past_values = f"""SELECT
    Series_Identifier,
    {time_column},
    {ATTRIBUTE_STRING},
    {target_column},
    'actual' AS {target_column}_Type
  FROM `{BQ_DATASET_NAME}.{BQ_TRAIN_TABLE}`
  WHERE
    {" IS NOT NULL AND ".join(ATTRIBUTE_COLUMNS)} IS NOT NULL"""


predicton_table_query = f"""WITH date_range AS (
  {date_range}
), columns_dim AS (
  {columns_dim}
),future_values AS (
  {future_values}
), past_values AS (
  {past_values}
)
SELECT
  Series_Identifier,
  {time_column},
  {ATTRIBUTE_STRING},
  {target_column},
  {target_column}_Type
FROM future_values
UNION ALL
SELECT
  Series_Identifier,
  {time_column},
  {ATTRIBUTE_STRING},
  {target_column},
  {target_column}_Type
FROM past_values
"""


In [ ]:
client.query_and_wait(f"""CREATE OR REPLACE TABLE `{BQ_DATASET_NAME}.{BQ_PREDICT_TABLE}` AS {predicton_table_query}""")

In [ ]:
batch_prediction_job = model.batch_predict(
    job_display_name=VERTEX_PREDICTION_NAME,
    bigquery_source=PREDICTION_DATASET_BQ_PATH,
    instances_format="bigquery",
    bigquery_destination_prefix=PREDICTION_OUTPUT_PREFIX,
    predictions_format="bigquery",
    generate_explanation=True,
    sync=True,
)

In [ ]:
#batch_table  = batch_prediction_job.output_info.bigquery_output_table
batch_table = 'predictions_2025_02_11T02_47_34_829Z_934'

In [44]:
prediction_data = f"""
SELECT
  CAST('{FORECAST_TIMESTAMP}' AS TIMESTAMP) AS Forecast_Date,
  CAST(Appointment_Day AS DATE) AS Appointment_Day,
  Series_Identifier,
  {ATTRIBUTE_STRING},
  predicted_SWT.value AS SWT
FROM
  `{BQ_DATASET_NAME}.{batch_table}`
WHERE
  SWT_Type = 'predicted'
"""


perisist_predictions_query = f"""
INSERT INTO `{BQ_DATASET_NAME}.{BQ_FORECAST_TABLE}`
(
  Model,
  Forecast_Date,
  Series_Identifier,
  Appointment_Day,
  {ATTRIBUTE_STRING},
  SWT
)
WITH prediction_data AS (
  {prediction_data}
)
SELECT DISTINCT
  '{running_experiment.model}' AS Model,
  Forecast_Date,
  Series_Identifier,
  Appointment_Day,
  {ATTRIBUTE_STRING},
  SWT
FROM prediction_data
"""

In [ ]:
client.query_and_wait(perisist_predictions_query)
client.query_and_wait(f"DROP TABLE `{BQ_DATASET_NAME}.{batch_table}`")

# Evaluation

In [93]:
import pandas as pd

class DataEvaluationPreprocessor:

    REQUIRED_COLUMNS = ['Appointment_Day', 'SWT', 'Product_Grp', 'Work_Order_Action_Grp', 'District', 'Region_Type']
    DATA_RANGE = pd.to_datetime([
            "2024-07-01", "2024-08-01", "2024-09-01",
            "2024-10-01", "2024-11-01", "2024-12-01"
        ])

    
    def __init__(self, data: pd.DataFrame) -> None:

        self.data = data.copy()

        self.__check_columns()
        self.__sort_columns()
        self.__ensure_all_months()
        self.__remove_lower_priority_tiers()


        self.data['Appointment_Day'] = pd.to_datetime(self.data['Appointment_Day'])

    def __sort_columns(self):
        self.data = self.data[self.data.columns.sort_values()]
        
    def get_filtered_data(self, filters: dict):
        """
        Apply a set of filters to a dataframe.
        Expected format for filters is a dict, e.g. {'Region_Type': 'Tier 1'}.
        """
        filtered_data = self.data.copy()
        for col, val in filters.items():
            filtered_data = filtered_data[filtered_data[col] == val]
        return filtered_data
    
    def get_grouped_data(self, group_by: list[str] = None, filters: dict = None):

        if group_by is None:
            group_by = self.data.columns.drop(['SWT', 'Appointment_Day'])

        grouped_data = self.data.copy()
        
        if filters is not None:
            grouped_data = self.get_filtered_data(filters)
            
        grouped_data['series_id'] = grouped_data[group_by].astype(str).agg(' '.join, axis=1)
        grouped_data = grouped_data[['Appointment_Day', 'series_id','SWT']]
        grouped_data.set_index('Appointment_Day', inplace=True)
        grouped_data = grouped_data.sort_values(by=['Appointment_Day', 'series_id'])
        grouped_data = grouped_data.groupby(['Appointment_Day', 'series_id'], as_index=True)['SWT'].sum()
        return grouped_data

    def __remove_lower_priority_tiers(self) -> pd.DataFrame:
        """
        Among rows that match exactly on all columns except 'Region_Type',
        keep only the row with the smallest numeric tier (e.g. Tier 1 < Tier 2 < Tier 3).
        """
        tier_map = {"Tier 1": 1, "Tier 2": 2, "Tier 3": 3, "Tier 4": 4}
        df = self.data.copy()
        df["tier_rank"] = df["Region_Type"].map(tier_map)
        
        # 2) Sort by all columns that define a duplicate plus the numeric tier rank.
        #    For duplicates, the smallest tier_rank will come first in sort order.
        sort_cols = [c for c in df.columns if c not in ["Region_Type", "tier_rank"]]
        df.sort_values(by=sort_cols + ["tier_rank"], inplace=True, ignore_index=True)
        
        # 3) Drop duplicates on every column **except** "Region_Type" (and our temporary "tier_rank")
        #    This means we only keep the first row of each group (the smallest tier).
        dedup_cols = [c for c in df.columns if c not in ["Region_Type", "tier_rank"]]
        df.drop_duplicates(subset=dedup_cols, keep="first", inplace=True, ignore_index=True)
        
        # 4) Drop the helper column
        df.drop(columns=["tier_rank"], inplace=True)
        
        self.data = df.copy()
        return df       

    def __check_columns(self):
        """
            Check if the df has this columns:
            - Appointment_Day [date]
            - SWT [float]
            - Product_Grp [str]
            - Work_Order_Action_Grp [str]
            - District [str]
            - Region_Type [str]
        """
        
        missing_columns = set(self.REQUIRED_COLUMNS) - set(self.data.columns)
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")  

        # cast the data for the right types
        self.data['Appointment_Day'] = pd.to_datetime(self.data['Appointment_Day'])
        self.data['SWT'] = self.data['SWT'].astype(float)
        self.data['Product_Grp'] = self.data['Product_Grp'].astype(str)
        self.data['Work_Order_Action_Grp'] = self.data['Work_Order_Action_Grp'].astype(str)
        self.data['District'] = self.data['District'].astype(str)
        self.data['Region_Type'] = self.data['Region_Type'].astype(str)
            
    def __has_all_months(self):
        """
        Check if the df has all months from 2024-07-01 to 2024-12-01 for each series_id
        """
        data = self.data.copy()
        data['series_id'] = data[data.columns.drop(['SWT', 'Appointment_Day'])].astype(str).agg(' '.join, axis=1)
        series_ids = data['series_id'].unique()
        for series_id in series_ids:
            series_data = data[data['series_id'] == series_id]
            if len(series_data) != len(self.DATA_RANGE):
                return False
        return True

    def __ensure_all_months(self):
        """
        For every unique combination of columns (other than Appointment_Day and SWT),
        this function adds rows for all months between 2024-07-01 and 2024-12-01
        (inclusive) if they're missing. The SWT column is filled with 0 where data 
        does not exist in the original dataframe.
        """

        if self.__has_all_months():
            return self.data
        
        df = self.data.copy()
        df['Appointment_Day'] = pd.to_datetime(df['Appointment_Day'])

        id_cols = [col for col in df.columns if col not in ['Appointment_Day', 'SWT']]
        unique_ids = df[id_cols].drop_duplicates()
        months_df = pd.DataFrame({'Appointment_Day': self.DATA_RANGE})

        unique_ids['merge_key'] = 1
        months_df['merge_key'] = 1
        cross_joined = pd.merge(unique_ids, months_df, on='merge_key').drop(columns='merge_key')

        merged = pd.merge(
            cross_joined,
            df,
            on=id_cols + ['Appointment_Day'],
            how='left'
        )

        merged['SWT'] = merged['SWT'].fillna(0)

        merged = merged.sort_values(by=['Appointment_Day'] + id_cols).reset_index(drop=True)

        merged = merged[['Appointment_Day'] + sorted(set(merged.columns) - {'Appointment_Day', 'SWT'}) + ['SWT']]

        self.data = merged.copy()
        return merged
    

import pandas as pd
import numpy as np


class Evaluation:
    
    def __init__(self, historical: DataEvaluationPreprocessor, predicted: DataEvaluationPreprocessor) -> None:
        """
        Initialize with two DataEvaluationPreprocessor objects: one for historical (actual) data
        and one for predicted (forecast) data.
        """
        self.historical = historical
        self.predicted = predicted

    def _get_data(
        self, 
        group_by: list[str] = None, 
        filters: dict = None
    ) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        Retrieve historical and predicted data (as DataFrames), optionally filtered and/or grouped.
        
        - If filters is provided (as a dict of {column: value}), the data are filtered.
        - If group is True, data are grouped (default grouping is by ['series_id', 'Appointment_Month'] 
          if no group_by is provided). In grouping, the SWT values are summed.
        """
        # Reset index to get Appointment_Month as a column.
        hist = self.historical.get_grouped_data(group_by, filters)
        pred = self.predicted.get_grouped_data(group_by, filters)

        return hist, pred

    def calculate_metric(
        self, 
        metric: str, 
        filters: dict = None, 
        group_by: list[str] = None, 
        epsilon: float = 1
    ) -> float:
        """
        Calculate a metric between the historical and predicted data.
        
        Parameters:
           metric:    str - Supported metrics: 'rmse', 'mape', or 'wape'
           group:     bool - Whether to group the raw data before merging.
                            If True, the data is grouped (default group_by = ['series_id', 'Appointment_Day']).
           filters:   dict - Optional filtering criteria (e.g. {'Region_Type': 'Tier 1'}).
           group_by:  list[str] - The columns to group by if group is True.
           epsilon:   float - A small value to avoid division by zero in percentage calculations.
        
        Returns:
           The computed metric as a float.
        """
        hist, pred = self._get_data(group_by, filters)
        
        merged = pd.merge(
            hist.reset_index(), pred.reset_index(), 
            on=['Appointment_Day', 'series_id'], 
            suffixes=('_hist', '_pred'), 
            how='inner'
        )
        
        error = merged['SWT_pred'] - merged['SWT_hist']
        
        if metric.lower() == 'rmse':
            return np.sqrt(np.mean(error ** 2))
        elif metric.lower() == 'mape':
            return np.mean(np.abs(error) / (merged['SWT_hist'] + epsilon)) * 100
        elif metric.lower() == 'wape':
            return np.sum(np.abs(error)) / (np.sum(merged['SWT_hist']) + epsilon)
        else:
            raise ValueError(f"Unsupported metric: {metric}")

In [61]:
forecast_query = f"""
SELECT
  DATE_TRUNC(Appointment_Day, MONTH) AS Appointment_Day,
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type,
  SUM({target_column}) as SWT
FROM `{BQ_DATASET_NAME}.{BQ_FORECAST_TABLE}`
WHERE 
  Model = '{running_experiment.model}'
  AND Forecast_Date = CAST('{FORECAST_TIMESTAMP}' AS TIMESTAMP)
GROUP BY
  DATE_TRUNC({time_column}, MONTH),
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type
ORDER BY
  {time_column},
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type
"""

In [62]:
forecast_df = client.query_and_wait(forecast_query).to_dataframe()

In [67]:
historical_query = f"""
WITH historical_table AS (
  SELECT 
    DATE_TRUNC({time_column}, MONTH) AS Appointment_Day,
    Product_Grp,
    Work_Order_Action_Grp,
    District,
    Region_Type,
    SUM({target_column}) AS SWT
  FROM `{BQ_DATASET_NAME}.vw_wf_historical`
  WHERE {time_column} >= {TRAIN_TEST_DATA_SPLIT}
  GROUP BY 
    DATE_TRUNC({time_column}, MONTH),
    Product_Grp,
    Work_Order_Action_Grp,
    District,
    Region_Type
)
SELECT
  Appointment_Day,
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type,
  SWT
FROM historical_table
ORDER BY
  Appointment_Day,
  Product_Grp,
  Work_Order_Action_Grp,
  District,
  Region_Type
"""

In [68]:
historical_df = client.query_and_wait(historical_query).to_dataframe()


In [94]:
historical_data = DataEvaluationPreprocessor(historical_df)
forecast_data = DataEvaluationPreprocessor(forecast_df)

In [95]:
evaluation = Evaluation(historical_data, forecast_data)

rmse = {
    'overall': evaluation.calculate_metric('rmse'),
    'Tier 1': evaluation.calculate_metric('rmse', filters={'Region_Type': 'Tier 1'}),
    'Tier 2': evaluation.calculate_metric('rmse', filters={'Region_Type': 'Tier 2'}),
    'Tier 3': evaluation.calculate_metric('rmse', filters={'Region_Type': 'Tier 3'})
}

wape = {
    'overall': evaluation.calculate_metric('wape'),
    'Tier 1': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 1'}),
    'Tier 2': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 2'}),
    'Tier 3': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 3'})
}

In [98]:
print(rmse, wape)

{'overall': 97690.4974779541, 'Tier 1': 152945.9169026087, 'Tier 2': 72308.02973904515, 'Tier 3': 17695.67033368956} {'overall': 0.7180642741536833, 'Tier 1': 0.7201755315924308, 'Tier 2': 0.7182299956441255, 'Tier 3': 0.6980604280223551}


In [99]:
evaluation_insert_query = f"""
INSERT INTO `{BQ_DATASET_NAME}.bq_wf_evaluation`
  (Model, Forecast_Date, WAPE, RMSE)
VALUES (
  '{running_experiment.model}',               
  '{FORECAST_TIMESTAMP}',
  STRUCT(
    {wape['overall']} AS Overall,
    {wape['Tier 1']} AS Tier_1,
    {wape['Tier 2']} AS Tier_2,
    {wape['Tier 3']} AS Tier_3
  ),
  STRUCT(
    {rmse['overall']} AS Overall,
    {rmse['Tier 1']} AS Tier_1,
    {rmse['Tier 2']} AS Tier_2,
    {rmse['Tier 3']} AS Tier_3
  )
)
"""

In [101]:
client.query_and_wait(evaluation_insert_query)